In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io as io
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
#对读入的训练csv进行有效数据拆分，表头重命名,
raw_data_train=pd.read_csv(r"C:\Users\fdl\Desktop\AML\cartoon_set\labels.csv",names=["mixed_information_train"])
data_unmixed_train = raw_data_train["mixed_information_train"].str.split("\t",expand=True)
data_unmixed_train.columns=['index','eye_color','face_shape','file_name']
data_unmixed_train.drop(columns=['index','eye_color','file_name'],inplace=True)
data_unmixed_train.drop(index=0,inplace=True)
#data_unmixed_train
#对读入的测试集结果进行相同的处理
raw_data_test=pd.read_csv(r"C:\Users\fdl\Desktop\AML\dataset_AMLS_22-23_test\cartoon_set_test\labels.csv",names=["mixed_information_test"])
data_unmixed_test = raw_data_test["mixed_information_test"].str.split("\t",expand=True)
data_unmixed_test.columns=['index','eye_color','face_shape','file_name']
data_unmixed_test.drop(columns=['index','eye_color','file_name'],inplace=True)
data_unmixed_test.drop(index=0,inplace=True)
#data_unmixed_test
raw_data_newlabel=pd.read_csv(r"C:\Users\fdl\Desktop\AML\cartoon_set\Lable.csv",names=["sunglass"])
#raw_data_newlabel.head()

In [3]:
#将训练和测试的结果存储为数组
YTrain=data_unmixed_train
YTest=data_unmixed_test
YTrain=YTrain.values
YTest=YTest.values
YTrain=np.array([int(y) for y in YTrain])
YTest=np.array([int(y) for y in YTest])
YTrain=YTrain.reshape(len(YTrain),1)
YTest=YTest.reshape(len(YTest),1)
#np.array(YTrain).shape,np.array(YTest).shape

In [4]:
XTrain=[]
XTest=[]
str_train= r"C:\Users\fdl\Desktop\AML\cartoon_set\img\*.png"
str_test= r"C:\Users\fdl\Desktop\AML\dataset_AMLS_22-23_test\cartoon_set_test\img\*.png"
coll_train = io.ImageCollection(str_train)
coll_test = io.ImageCollection(str_test)
for img in coll_train:
    intern_train=cv2.resize(img, (100,100))
    XTrain.append(intern_train)
XTrain=np.array(XTrain)/255
for img in coll_test:
    intern_test=cv2.resize(img, (100,100))
    XTest.append(intern_test)
XTest=np.array(XTest)/255

In [ ]:
conv_num=[3]
fully_connected_layer_num=[3]
unit_count=[100]
for conv in conv_num:
    for fully_connected_layer in fully_connected_layer_num:
        for unit in unit_count:
            logs_file=f'logs/log_{conv}_fully_{fully_connected_layer}_unit_{unit}_conv_{int(time.time())}'
            tensorboard=TensorBoard(log_dir=logs_file)
            #checkpoint
            checkpoint_filepath=f'models/model_{conv}_fully_{fully_connected_layer}_unit_{unit}_'+'{epoch:02d}-{val_accuracy:.2f}.hdf5'
            print(logs_file)
            print(checkpoint_filepath)
            checkpoint=ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',model='max',save_best_only=True,verbose=1)
            model=Sequential()
            
            #convloution layer
            model.add(Conv2D(unit,(5,5),activation='relu'))
            #Maxpooling layer
            model.add(MaxPooling2D(pool_size=(3,3)))
            for i in range(conv-1):
                #convloution layer
                model.add(Conv2D(unit,(5,5),activation='relu'))
                model.add(MaxPooling2D(pool_size=(3,3)))
            #Flatten layer
            model.add(Flatten())
            #fully connected layer
            for i in range(fully_connected_layer):
                model.add(Dense(unit,activation='relu'))
            #output layer
            model.add(Dense(5,activation='softmax'))
            #compile
            model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
            #fit
            model.fit(XTrain,YTrain,batch_size=32,epochs=5,validation_split=0.1,callbacks=[tensorboard])

logs/log_3_fully_3_unit_100_conv_1672686358
models/model_3_fully_3_unit_100_{epoch:02d}-{val_accuracy:.2f}.hdf5
Epoch 1/5
282/282 [==============================] - 51s 177ms/step - loss: 1.2689 - accuracy: 0.3972 - val_loss: 0.1034 - val_accuracy: 0.9790
Epoch 2/5
282/282 [==============================] - 51s 182ms/step - loss: 0.0642 - accuracy: 0.9793 - val_loss: 0.0209 - val_accuracy: 0.9960
Epoch 3/5
282/282 [==============================] - 52s 184ms/step - loss: 0.0181 - accuracy: 0.9940 - val_loss: 0.0088 - val_accuracy: 0.9970
Epoch 4/5
265/282 [===========================>..] - ETA: 3s - loss: 0.0145 - accuracy: 0.9955

In [73]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (100, 48, 48, 100)        3700      
                                                                 
 max_pooling2d_20 (MaxPoolin  (100, 24, 24, 100)       0         
 g2D)                                                            
                                                                 
 flatten_19 (Flatten)        (100, 57600)              0         
                                                                 
 dense_40 (Dense)            (100, 100)                5760100   
                                                                 
 dense_41 (Dense)            (100, 5)                  505       
                                                                 
Total params: 5,764,305
Trainable params: 5,764,305
Non-trainable params: 0
___________________________________________